# MACHINE LEARNING MODELS - FEATURE IMPORTANCE

In this notebook, we specifically examin feature importance of the various models.

In [1]:
# Import libraries and functions

#Data Science usual libraries
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

# Scikit-learn libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.inspection import permutation_importance

#OverSampling and UnderSampling
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

### Classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# define random_state
RANDOM_STATE = 3
n_repeats = 30
%matplotlib inline

### 1. Load data

We load our csv into a dataframe using pandas.

In [2]:
## Define paths

dataDir = "data/"
singleLayerFile = "single_layer.csv"
multiLayerFile = "multi_layer.csv"

singleLayer = pd.read_csv(dataDir + singleLayerFile)
multiLayer = pd.read_csv(dataDir + multiLayerFile)

In [3]:
print(singleLayer.shape)
print(multiLayer.shape)

(165, 22)
(165, 38)


In [4]:
list(singleLayer)

['id',
 'controls_ms',
 'mstype',
 'age',
 'sex',
 'dd',
 'edss',
 'FA_diameter',
 'fMRI_diameter',
 'fMRI_gl-efficiency',
 'fMRI_mean_path-length',
 'fMRI_gl-transitivity',
 'fMRI_density',
 'FA_loc-clos-cent_ctx-lh-isthmuscingulate',
 'FA_loc-clos-cent_Left-Hippocampus',
 'FA_loc-clos-cent_ctx-rh-caudalanteriorcingulate',
 'FA_loc-clos-cent_ctx-rh-lateralorbitofrontal',
 'FA_loc-clos-cent_ctx-rh-precentral',
 'FA_loc-strength_Right-Caudate',
 'fMRI_loc-clos-cent_ctx-rh-entorhinal',
 'fMRI_loc-clos-cent_ctx-rh-parahippocampal',
 'fMRI_loc-clos-cent_ctx-rh-paracentral']

In [5]:
singleLayer.head()

,id,controls_ms,mstype,age,sex,dd,edss,FA_diameter,fMRI_diameter,fMRI_gl-efficiency,...,fMRI_density,FA_loc-clos-cent_ctx-lh-isthmuscingulate,FA_loc-clos-cent_Left-Hippocampus,FA_loc-clos-cent_ctx-rh-caudalanteriorcingulate,FA_loc-clos-cent_ctx-rh-lateralorbitofrontal,FA_loc-clos-cent_ctx-rh-precentral,FA_loc-strength_Right-Caudate,fMRI_loc-clos-cent_ctx-rh-entorhinal,fMRI_loc-clos-cent_ctx-rh-parahippocampal,fMRI_loc-clos-cent_ctx-rh-paracentral
0,002MSVIS,PwMS,SPMS,51.53,Male,22.18,7.5,0.332354,0.065675,93.889111,...,0.974737,0.064372,0.096570,0.077665,0.079250,0.074236,10.638695,0.603022,0.537120,0.559320
1,003MSVIS,PwMS,PPMS,58.06,Female,9.57,6.0,0.301074,0.075368,55.937286,...,0.966316,0.068736,0.079853,0.082105,0.071313,0.079974,9.515183,0.373515,0.391899,0.570189
2,004MSVIS,PwMS,RRMS,60.78,Male,6.79,3.0,0.306527,0.087877,61.506756,...,0.958947,0.071388,0.098190,0.071972,0.079739,0.080090,9.860426,0.406553,0.554453,0.648328
3,005MSVIS,PwMS,RRMS,37.95,Female,8.45,1.5,0.360501,0.066865,82.930529,...,0.976140,0.064019,0.077450,0.070209,0.074123,0.064426,8.655474,0.467528,0.890219,0.487918
4,010MSVIS,PwMS,RRMS,52.64,Female,9.05,2.0,0.342449,0.065693,61.978222,...,0.963158,0.065071,0.080101,0.069243,0.068634,0.072694,10.569179,0.668055,0.517563,0.839344


In [6]:
multiLayer.head()

,id,controls_ms,mstype,age,sex,dd,edss,MultiPRCent_ctx-lh-lingual,MultiPRCent_ctx-lh-precentral,MultiPRCent_Left-Thalamus-Proper,...,MultiSTSum_ctx-rh-lingual,MultiSTSum_ctx-rh-superiorfrontal,MultiSTSum_ctx-rh-superiortemporal,MultiPath_ctx-lh-lateraloccipital,MultiPath_ctx-lh-pericalcarine,MultiPath_ctx-lh-precuneus,MultiPath_ctx-lh-supramarginal,MultiPath_Left-Accumbens-area,MultiPath_Right-Accumbens-area,MultiPath_ctx-rh-superiorparietal
0,002MSVIS,PwMS,SPMS,51.53,Male,22.18,7.5,0.509232,0.805250,0.616926,...,28.649804,56.195174,40.722663,77.795450,113.733167,64.186044,52.703518,689.959892,230.378121,51.470548
1,003MSVIS,PwMS,PPMS,58.06,Female,9.57,6.0,0.590916,0.782418,0.656285,...,32.017065,53.267982,42.426642,46.551148,83.524268,40.299180,40.751312,109.436556,157.131119,39.262600
2,004MSVIS,PwMS,RRMS,60.78,Male,6.79,3.0,0.680297,0.884523,0.651378,...,34.934002,50.668714,45.193010,79.384546,61.563700,74.681403,31.225727,104.659071,128.412529,45.468482
3,005MSVIS,PwMS,RRMS,37.95,Female,8.45,1.5,0.526728,0.792428,0.630885,...,32.448897,55.010716,44.606681,37.883356,91.516103,47.070550,88.923005,107.904849,139.792145,102.843536
4,010MSVIS,PwMS,RRMS,52.64,Female,9.05,2.0,0.604058,0.727934,0.603382,...,34.260856,54.705097,43.989355,40.897744,79.886554,121.280869,56.496038,102.811904,149.682831,91.088859


In [7]:
# Concatenate data frames to gather all data

all_data = pd.concat([singleLayer, multiLayer.iloc[:, 7:]], join = "inner", axis = 1)

all_data.shape

(165, 53)

## 2. Remove highly correlated data

Graph measures could exhibit high correlation (for example, diameter measures and mean path length, different centrality measures, and so on). Most machine learning algorithms perform better without highly correlated features. Therefore, we will remove any columns that are significantly correlated with others, retaining only one representative measure from each set of correlated measures.

We set our threshold at 0.8, accounting for both positive and negative correlations, and remove all correlated columns over (or below) threshold.

In [8]:
selected_columns = all_data.iloc[:, 7:] 
colnames = list(selected_columns)

pos_threshold = 0.8
neg_threshold = -0.8

highly_correlated_columns = {} #Dictionary to store correlations
columns_to_remove = []
for i,colname in enumerate(colnames):
    columns_to_dict = [] # Dictionary values
    if selected_columns.columns[i] not in columns_to_remove: #Column not in the list to remove
        for j in range(i+1, len(colnames)):
            if selected_columns.columns[j] not in columns_to_remove: #Column not in the list to remove
                matrix_coef = np.corrcoef(selected_columns.iloc[:, i], selected_columns.iloc[:, j]) 
                if matrix_coef[0,1] > pos_threshold or matrix_coef[0,1] < neg_threshold:
                    columns_to_remove.append(selected_columns.columns[j])
                    columns_to_dict.append(selected_columns.columns[j])
        highly_correlated_columns[colname] = columns_to_dict

columns_to_remove = set(columns_to_remove) #just in case we have some repeated columns
print("Can remove " +str(len(columns_to_remove)) + " features")

# Select column that are not in the list
filtered_columns = [col for col in selected_columns.columns if col not in columns_to_remove] 
filtered_df = selected_columns[filtered_columns]

# Final dataframe
final_data = pd.concat([all_data.iloc[:, :7], filtered_df], join = "inner", axis = 1)
final_data.head()


Can remove 9 features


,id,controls_ms,mstype,age,sex,dd,edss,FA_diameter,fMRI_diameter,fMRI_gl-efficiency,...,MultiSTSum_Right-Thalamus-Proper,MultiSTSum_ctx-rh-inferiorparietal,MultiSTSum_ctx-rh-superiorfrontal,MultiSTSum_ctx-rh-superiortemporal,MultiPath_ctx-lh-lateraloccipital,MultiPath_ctx-lh-pericalcarine,MultiPath_ctx-lh-precuneus,MultiPath_ctx-lh-supramarginal,MultiPath_Left-Accumbens-area,MultiPath_ctx-rh-superiorparietal
0,002MSVIS,PwMS,SPMS,51.53,Male,22.18,7.5,0.332354,0.065675,93.889111,...,30.731027,34.974737,56.195174,40.722663,77.795450,113.733167,64.186044,52.703518,689.959892,51.470548
1,003MSVIS,PwMS,PPMS,58.06,Female,9.57,6.0,0.301074,0.075368,55.937286,...,35.844303,38.200251,53.267982,42.426642,46.551148,83.524268,40.299180,40.751312,109.436556,39.262600
2,004MSVIS,PwMS,RRMS,60.78,Male,6.79,3.0,0.306527,0.087877,61.506756,...,36.745742,35.645762,50.668714,45.193010,79.384546,61.563700,74.681403,31.225727,104.659071,45.468482
3,005MSVIS,PwMS,RRMS,37.95,Female,8.45,1.5,0.360501,0.066865,82.930529,...,34.118122,36.883135,55.010716,44.606681,37.883356,91.516103,47.070550,88.923005,107.904849,102.843536
4,010MSVIS,PwMS,RRMS,52.64,Female,9.05,2.0,0.342449,0.065693,61.978222,...,35.323594,36.149344,54.705097,43.989355,40.897744,79.886554,121.280869,56.496038,102.811904,91.088859


In [9]:
final_data.shape

(165, 44)

In [10]:
len_correlations ={key:len(highly_correlated_columns[key]) for key in highly_correlated_columns.keys()}
len_correlations

{'FA_diameter': 0,
 'fMRI_diameter': 1,
 'fMRI_gl-efficiency': 0,
 'fMRI_gl-transitivity': 1,
 'FA_loc-clos-cent_ctx-lh-isthmuscingulate': 0,
 'FA_loc-clos-cent_Left-Hippocampus': 1,
 'FA_loc-clos-cent_ctx-rh-caudalanteriorcingulate': 0,
 'FA_loc-clos-cent_ctx-rh-lateralorbitofrontal': 0,
 'FA_loc-strength_Right-Caudate': 0,
 'fMRI_loc-clos-cent_ctx-rh-entorhinal': 0,
 'fMRI_loc-clos-cent_ctx-rh-parahippocampal': 0,
 'fMRI_loc-clos-cent_ctx-rh-paracentral': 0,
 'MultiPRCent_ctx-lh-lingual': 0,
 'MultiPRCent_ctx-lh-precentral': 0,
 'MultiPRCent_Left-Thalamus-Proper': 1,
 'MultiPRCent_Left-Caudate': 0,
 'MultiPRCent_Left-Putamen': 0,
 'MultiPRCent_Left-Hippocampus': 0,
 'MultiPRCent_Left-Amygdala': 0,
 'MultiPRCent_Left-Accumbens-area': 1,
 'MultiPRCent_Right-Caudate': 1,
 'MultiPRCent_Right-Putamen': 0,
 'MultiPRCent_Right-Accumbens-area': 1,
 'MultiPRCent_ctx-rh-lateraloccipital': 0,
 'MultiPRCent_ctx-rh-lingual': 1,
 'MultiPRCent_ctx-rh-precentral': 0,
 'MultiSTSum_ctx-lh-middletempor

## 2. Define functions

We define some functions we will use throught the notebook

It is worth noting that in our pipepline needs scaling (normalize) our values, as SMOTE and machine learning models perform better with normalized values



In [11]:
def make_pipeline(model, num_neighbors = 5):
    """
    Function that creates a pipeline with Standard Scaler, SMOTE and model selected
    INPUT 
    model --> a scikit model like RandomForestClassifier()
    num_neighbors --> an int, default 5. Indicates number of neighbors SMOTE will use
    
    Returns 
    a pipeline from imblearn
    """
    over = SMOTE(random_state = RANDOM_STATE, k_neighbors= num_neighbors)    
    sc = StandardScaler()
    model = model
    steps = [('Scaler', sc), ('over', over), ('model', model)]
    pipeline = Pipeline(steps = steps)
    
    return pipeline

def search_grid(model, param_grid, num_neighbors = 5):
    """
    Function to perfrom GridSearchCV on a model.
    
    INPUT: 
    model --> a scikit model like RandomForestClassifier()
    param_grid --> a dict with parameters to perform GridSearchCV
    num_neighbors --> an int, default 5. Indicates number of neighbors SMOTE will use
    
    CALLS make_pripeline 
    
    PRINTS  GridSearchCV best parameters
    
    RETURNS GridSearchCV result
    
    """
  # Depending if we are doing a binary classification (HS - PWMS) o multiclass (different MStypes)
  # we choose a scoring type
    if len(le_name_mapping.keys()) > 2:
        scoring = "f1_weighted"
    else:
        scoring = "f1"

    #Perform Grid Search    
    GridSearch_model = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5, scoring = scoring)
    pipeline = make_pipeline(GridSearch_model, num_neighbors = num_neighbors)
    pipeline.fit(X_train, y_train) 
    
    
    print(GridSearch_model.best_params_)
    
    return GridSearch_model

def plot_hist_GridSearch(means, model_name):
    """
    Function to plot a histogram from a list of means that represent Mean Test Scores 
    from GridSearchCV
    INPUT
    means --> a list of numbers
    model_name <- a string to identify model
    """
    #plt.hist(scores, bins = 10)
    sns.histplot(means, bins=10)
    xmin = min(means)
    if xmin > 0.9:
        xmin = 0.9
    plt.xlim(xmin, 1)
    plt.xlabel('Score')
    plt.ylabel('Count')
    plt.title(model_name, fontsize = 18)

In [12]:
def perm_impor_bar(model, model_name, classification = "HS vs PwMS"):
    if len(le_name_mapping.keys()) > 2:
        scoring = "f1_weighted"
    else:
        scoring = "f1"
    results = permutation_importance(model, X_test, y_test, scoring = scoring, n_repeats=n_repeats,
                               random_state= RANDOM_STATE)
    
    plot_title = '3-Multilayer Permutation importace '+model_name+ ' '+classification
    file ="images/3_Multi_perm_imp_bar_"+classification+"_"+model_name+".png"
    
    perm = pd.DataFrame(columns=['AVG_Importance', 'STD_Importance'], 
                        index=[i for i in X.columns])
    perm['AVG_Importance'] = results.importances_mean
    perm['STD_Importance'] =results.importances_std
    
  
    
    perm = perm.sort_values('AVG_Importance', ascending = False)
    
    
    
    perm = perm[perm['AVG_Importance'] != 0] #Remove 0s
    
    valueTop = perm.iloc[0,0] -perm.iloc[1,0]
    valueBottom = perm.iloc[-1,0] - perm.iloc[-2,0]
    
    #Create colors
    if abs(valueTop) < 0.001:
        perm['color'] = ['blue' if (i == 0 or i==1)  else 'red' if i == len(perm) -1 else 'orange' for i in range(len(perm))]
    elif abs(valueBottom) < 0.001:
        perm['color'] = ['blue' if i == 0  else 'red' if (i == len(perm) -1 or i == len(perm) -2 ) else 'orange' for i in range(len(perm))]
    else:
        perm['color'] = ['blue' if i == 0  else 'red' if i == len(perm) -1 else 'orange' for i in range(len(perm))]

    if perm.empty:
        print("All features are zero")
    else:
        plt.figure(figsize=(14, 10))
        sns.barplot(x = perm['AVG_Importance'], y = perm.index, palette = perm['color'] )
        plt.title(plot_title, fontsize = 20)
        plt.xlabel("Average of Importance.", fontsize = 16)
        plt.tick_params(axis='y', labelsize=16)
        plt.savefig(file, bbox_inches='tight')
        plt.show()
        return(perm)    


## 3. BINARY CLASSIFICATION 
## Healthy Subjects (HS) and Patients with Multiple Sclerosis (PwMS)

In this section, we will deploy machine learning models to attempt to distinguish between healthy subjects and patients with Multiple Sclerosis, without differentiating patients by their specific illness phenotype

First we divide our dataset in train and test. 

In [13]:
X = final_data.iloc[:,7:]
y = final_data["controls_ms"]

# Our values are encoded in labels rather than in 0s and 1s, so we use label encoder
le = LabelEncoder()
y = le.fit_transform(y)

#We need to make a change to use permutation_importance
features = X.columns
X_perm = X.values

# Create a dictionary with correspondences between labels and encoded labels
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_))) 


X_train, X_test, y_train, y_test = train_test_split(X_perm, y, test_size=0.2, random_state= RANDOM_STATE)

### 3.1. RANDOM FOREST FOR BINARY CLASSIFICATION

In [14]:
param_grid = {
    'n_estimators': [25, 50, 75, 100, 150],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [2, 5, 7, 10, 15]
}

GridSearch_RF = search_grid(RandomForestClassifier(random_state = RANDOM_STATE), param_grid)
max_depth = GridSearch_RF.best_params_['max_depth']
max_features = GridSearch_RF.best_params_['max_features']
n_estimators = GridSearch_RF.best_params_['n_estimators']
model = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, 
                               max_features = max_features, 
                               random_state = RANDOM_STATE)
pipeline = make_pipeline(model, num_neighbors = 5)
pipeline.fit(X_train, y_train)


{'max_depth': 7, 'max_features': 'log2', 'n_estimators': 75}


Pipeline(steps=[('Scaler', StandardScaler()), ('over', SMOTE(random_state=3)),
                ('model',
                 RandomForestClassifier(max_depth=7, max_features='log2',
                                        n_estimators=75, random_state=3))])

In [15]:
test_rf_bar = perm_impor_bar(pipeline, "RF", "Binary")

In [16]:
test_rf_bar

,AVG_Importance,STD_Importance,color
MultiPRCent_Left-Accumbens-area,0.007897,0.008442,blue
MultiSTSum_ctx-rh-inferiorparietal,0.005148,0.011549,orange
MultiPath_ctx-lh-pericalcarine,0.005008,0.010571,orange
MultiPRCent_Left-Hippocampus,0.001128,0.004221,orange
fMRI_gl-efficiency,0.000651,0.010787,orange
fMRI_loc-clos-cent_ctx-rh-parahippocampal,-0.000546,0.002941,orange
fMRI_gl-transitivity,-0.001021,0.009531,orange
fMRI_loc-clos-cent_ctx-rh-entorhinal,-0.001639,0.004916,orange
MultiPath_ctx-lh-lateraloccipital,-0.002185,0.005570,orange
MultiPRCent_Right-Accumbens-area,-0.002793,0.011371,orange


### 3.2. XGBOOST FOR BINARY CLASSIFICATION

In [17]:
param_grid =[{"learning_rate": [0.20, 0.25, 0.30],
              "max_depth": [4, 5, 6, 7],
                     "min_child_weight": [ 1, 3],
                     "gamma": [0.0, 0.05, 0.1, 0.2],
                     "colsample_bytree": [0.3, 0.4, 0.5, 0.7],
                     "verbosity": [0]}]# add this line to slient warning message
importance_list = []


GridSearch_XGB = search_grid(XGBClassifier(random_state = RANDOM_STATE), param_grid)
learning_rate = GridSearch_XGB.best_params_["learning_rate"]
max_depth = GridSearch_XGB.best_params_["max_depth"]
min_child_weight =GridSearch_XGB.best_params_["min_child_weight"]
gamma = GridSearch_XGB.best_params_["gamma"]
colsample_bytree = GridSearch_XGB.best_params_["colsample_bytree"]
model = XGBClassifier(random_state = RANDOM_STATE,
                        learning_rate = learning_rate, 
                      max_depth = max_depth,
                     min_child_weight = min_child_weight,
                     gamma = gamma,
                     colsample_bytree = colsample_bytree,
                     verbosity = 0)
pipeline = make_pipeline(model, num_neighbors = 5)
pipeline.fit(X_train, y_train)

{'colsample_bytree': 0.3, 'gamma': 0.1, 'learning_rate': 0.25, 'max_depth': 4, 'min_child_weight': 1, 'verbosity': 0}


Pipeline(steps=[('Scaler', StandardScaler()), ('over', SMOTE(random_state=3)),
                ('model',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=0.3, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0.1, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.25,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=4, max_leaves=None, min_child_weight=1,
                               missing=nan, monotone_constraints=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, predictor=None,
                               random_state=3, ...))])

In [18]:
test_XGB_bar = perm_impor_bar(pipeline, "XGB", "Binary")